In [1]:
import os
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
import argparse
import itertools
import json
import pandas as pd
import pickle
from joblib import Parallel, delayed
import mne
import mne_bids
from mne.minimum_norm import apply_inverse,apply_inverse_epochs

subject_id="CB040"
visit_id="1"
bids_root="/headnode1/abry4213/data/Cogitate_MEG_challenge"
region_option="hypothesis_driven"
n_jobs=1

In [2]:
duration='1000ms'
subject_time_series_path=f"{bids_root}/derivatives/MEG_time_series/sub-{subject_id}/ses-{visit_id}/meg/"

In [9]:
time_series_files = [f"{subject_time_series_path}/{file}" for file in os.listdir(subject_time_series_path) if "1000ms_epoch" in file]
all_time_series_data = pd.concat([pd.read_csv(file) for file in time_series_files])

# Average across epochs
averaged_by_condition = (all_time_series_data
                         .groupby(["stimulus_type", "relevance_type", "duration", "times", "meta_ROI"], as_index=False)["data"]
                         .mean()
                         .assign(meta_ROI = lambda x: x.meta_ROI.str.replace("_meta_ROI", ""))
                         .pivot(index=["stimulus_type", "relevance_type", "duration", "times"], columns="meta_ROI", values="data"))

averaged_by_condition.columns = averaged_by_condition.columns.get_level_values(0)
averaged_by_condition.reset_index(inplace=True)

# Save to CSV
output_path = f"{bids_root}/derivatives/MEG_time_series/sub-{subject_id}_ses-{visit_id}_meg_{duration}_all_time_series.csv"
averaged_by_condition.to_csv(output_path, index=False)

meta_ROI,stimulus_type,relevance_type,duration,times,Category_Selective,GNWT,IIT
0,False,Irrelevant,1000ms,-0.500,0.032584,0.022722,-0.004934
1,False,Irrelevant,1000ms,-0.499,0.040036,0.027676,-0.004166
2,False,Irrelevant,1000ms,-0.498,0.054817,0.033102,0.001756
3,False,Irrelevant,1000ms,-0.497,0.018389,0.025569,0.003065
4,False,Irrelevant,1000ms,-0.496,-0.002571,0.015190,-0.006169


In [ ]:
# Helper function to create a dictionary of ROI labels depending on the type of region subset requested
def compute_ROI_labels(labels_atlas, region_option, rois_deriv_root):
    # Create dictionary to store labels and vertices
    labels_dict = {}
    if region_option == 'hypothesis_driven':
        # Read GNW and IIT ROI list
        f = open(op.join(rois_deriv_root,
                        'hypothesis_driven_ROIs.json'))
        hypothesis_driven_ROIs = json.load(f)

        # GNWT ROIs
        print("GNWT ROIs:")
        for lab in hypothesis_driven_ROIs['GNWT_ROIs']:
            print(lab)
            labels_dict["GNWT_"+lab] = np.sum([l for l in labels_atlas if lab in l.name])

        # IIT ROIs
        print("IIT ROIs")
        for lab in hypothesis_driven_ROIs['IIT_ROIs']:
            print(lab)
            labels_dict["IIT_"+lab] = np.sum([l for l in labels_atlas if lab in l.name])

        # Category-selective ROIs
        print("Category-selective ROIs:")
        for lab in hypothesis_driven_ROIs['Category_Selective_ROIs']:
            print(lab)
            labels_dict["Category_Selective_"+lab] = np.sum([l for l in labels_atlas if lab in l.name])

        # Merge all labels in a single one separatelly for GNW and IIT 
        labels_dict['GNWT_meta_ROI'] = np.sum([l for l_name, l in labels_dict.items() if 'GNWT' in l_name])
        labels_dict['IIT_meta_ROI'] = np.sum([l for l_name, l in labels_dict.items() if 'IIT' in l_name])
        labels_dict['Category_Selective_meta_ROI'] = np.sum([l for l_name, l in labels_dict.items() if 'Category_Selective' in l_name])

        # Only keep the meta-ROIs
        labels_dict = {k: v for k, v in labels_dict.items() if 'meta_ROI' in k}

    else:
        for label in labels_atlas: 
            label_name = label.name
            labels_dict[label_name] =  np.sum([label])

    return labels_dict


In [ ]:
# Set task
bids_task = 'dur'

# Read epoched data
bids_path_epo = mne_bids.BIDSPath(
    root=prep_deriv_root, 
    subject=subject_id,  
    datatype='meg',  
    task=bids_task,
    session=visit_id, 
    suffix='epo',
    extension='.fif',
    check=False)

# Use Desikan--Killiany atlas to compute dictionary of labels
labels_atlas = mne.read_labels_from_annot(
    "sub-"+subject_id, 
    parc='aparc.a2009s',
    subjects_dir=fs_deriv_root)
labels_dict = compute_ROI_labels(labels_atlas, region_option, rois_deriv_root)

In [ ]:
# Set task
bids_task = 'dur'

# Read epoched data
bids_path_epo = mne_bids.BIDSPath(
    root=prep_deriv_root, 
    subject=subject_id,  
    datatype='meg',  
    task=bids_task,
    session=visit_id, 
    suffix='epo',
    extension='.fif',
    check=False)

bids_path_epo_rs = mne_bids.BIDSPath(
    root=prep_deriv_root, 
    subject=subject_id,  
    datatype='meg',  
    task=bids_task,
    session=visit_id, 
    suffix='epo_rs',
    extension='.fif',
    check=False)

print("Loading epochs data")
epochs_all = mne.read_epochs(bids_path_epo.fpath, preload=True)

epochs_final = epochs_all

In [ ]:
# Run baseline correction
print("Running baseline correction")
b_tmin = -0.5
b_tmax = 0.
baseline = (b_tmin, b_tmax)
epochs_final.apply_baseline(baseline=baseline)

In [ ]:
# Load rank + covariance matrices
with open(f"{fwd_deriv_root}/sub-{subject_id}_ses-{visit_id}_task-{bids_task}_rank.pkl", 'rb') as f:
            rank = pickle.load(f)

with open(f"{fwd_deriv_root}/sub-{subject_id}_ses-{visit_id}_task-{bids_task}_common_cov.pkl", 'rb') as f:
            common_cov = pickle.load(f)

In [ ]:
# Read forward model
print("Reading forward model")
bids_path_fwd = bids_path_epo.copy().update(
        root=fwd_deriv_root,
        task=bids_task,
        suffix="surface_fwd",
        extension='.fif',
        check=False)
fwd = mne.read_forward_solution(bids_path_fwd.fpath)

In [ ]:
# Make inverse operator
inverse_operator = mne.minimum_norm.make_inverse_operator(
    epochs_final.info,
    fwd, 
    common_cov,
    loose=.2,
    depth=.8,
    fixed=False,
    rank=rank,
    use_cps=True)

In [ ]:
cond_combs = list(itertools.product(conditions[0],
                                    conditions[1],
                                    conditions[2]))


cond_comb = cond_combs[0]

epochs = epochs_final['%s == "%s" and %s == "%s" and %s == "%s"' % (
            factor[0], cond_comb[0], 
            factor[1], cond_comb[1], 
            factor[2], cond_comb[2])]
fname_base = f"{cond_comb[0]}_{cond_comb[1]}_{cond_comb[2]}".replace(" ","-")

In [ ]:
# Take just the first epoch
epoch_first = epochs_final[0]

# Compute inverse solution for each epoch
snr = 3.0
lambda2 = 1.0 / snr ** 2
stcs = apply_inverse_epochs(epoch_first, inverse_operator,
                            lambda2=1.0 / snr ** 2, verbose=False,
                            method="dSPM", pick_ori="normal")

# extract time course in label with pca_flip mode
src = inverse_operator['src']

# Extract time course for each stc
epoch_times_df_list = []
for i in range(len(stcs)):

    # Find epoch number
    epoch_number = i+1

    # Find stc
    stc = stcs[i]

    # Loop over labels        
    for label_name, label in labels_dict.items():

        # Select data in label
        stc_in = stc.in_label(label)

        # Extract time course data, averaged across channels within ROI
        times = stc_in.times
        data = stc_in.data.mean(axis=0)

        # Concatenate into dataframe
        epoch_df = pd.DataFrame({
            'epoch_number': epoch_number,
            'stimulus_type': cond_comb[0], 
            'relevance_type': cond_comb[1],
            'duration': cond_comb[2],
            'times': times,
            'meta_ROI': label_name,
            'data': data})
        
        # Write this epoch to a CSV file
        output_CSV_file = op.join(subject_time_series_output_path, f"{fname_base}_epoch{epoch_number}_{label_name}.csv")
        epoch_df.to_csv(output_CSV_file, index=False)

# Concatenate all the epoch times
# epoch_times_df = pd.concat(epoch_times_df_list)